<u><h1>Women Cloth Reviews Prediction with Multi Nominal Navive Bayes</h1></u>
<p>Multinomial Naïve Bayes algorithm is a probabilistic learning method that is mostly used in Natural Language Processing. It is also suitable for text classification with discrete feature. In this project it will be used to build a women cloth review prediction model.</p>

<h3>- Import Library<h3>

In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

<h3>- Import dataset<h3>

In [68]:
df = pd.read_csv(r"C:\Users\nadee\Downloads\Womens Clothing E-Commerce Reviews.csv\Womens Clothing E-Commerce Reviews.csv")

<h3>- Discribe Dataset<h3>

In [70]:
df.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Unnamed: 0               23486 non-null  int64 
 1   Clothing ID              23486 non-null  int64 
 2   Age                      23486 non-null  int64 
 3   Title                    19676 non-null  object
 4   Review Text              22641 non-null  object
 5   Rating                   23486 non-null  int64 
 6   Recommended IND          23486 non-null  int64 
 7   Positive Feedback Count  23486 non-null  int64 
 8   Division Name            23472 non-null  object
 9   Department Name          23472 non-null  object
 10  Class Name               23472 non-null  object
dtypes: int64(6), object(5)
memory usage: 2.0+ MB


In [72]:
df.describe()

,Unnamed: 0,Clothing ID,Age,Rating,Recommended IND,Positive Feedback Count
count,23486.000000,23486.000000,23486.000000,23486.000000,23486.000000,23486.000000
mean,11742.500000,918.118709,43.198544,4.196032,0.822362,2.535936
std,6779.968547,203.298980,12.279544,1.110031,0.382216,5.702202
min,0.000000,0.000000,18.000000,1.000000,0.000000,0.000000
25%,5871.250000,861.000000,34.000000,4.000000,1.000000,0.000000
50%,11742.500000,936.000000,41.000000,5.000000,1.000000,1.000000
75%,17613.750000,1078.000000,52.000000,5.000000,1.000000,3.000000
max,23485.000000,1205.000000,99.000000,5.000000,1.000000,122.000000


In [73]:
df.columns

Index(['Unnamed: 0', 'Clothing ID', 'Age', 'Title', 'Review Text', 'Rating',
       'Recommended IND', 'Positive Feedback Count', 'Division Name',
       'Department Name', 'Class Name'],
      dtype='object')

<h3>- Data Preprocessing</h3>

In [75]:
df.isna().sum()

Unnamed: 0                    0
Clothing ID                   0
Age                           0
Title                      3810
Review Text                 845
Rating                        0
Recommended IND               0
Positive Feedback Count       0
Division Name                14
Department Name              14
Class Name                   14
dtype: int64

In [76]:
df[df['Review Text']== ""] = np.NaN
df['Review Text'].fillna("No Review", inplace= True)

In [77]:
df['Review Text']

0        Absolutely wonderful - silky and sexy and comf...
1        Love this dress!  it's sooo pretty.  i happene...
2        I had such high hopes for this dress and reall...
3        I love, love, love this jumpsuit. it's fun, fl...
4        This shirt is very flattering to all due to th...
                               ...                        
23481    I was very happy to snag this dress at such a ...
23482    It reminds me of maternity clothes. soft, stre...
23483    This fit well, but the top was very see throug...
23484    I bought this dress for a wedding i have this ...
23485    This dress in a lovely platinum is feminine an...
Name: Review Text, Length: 23486, dtype: object

In [78]:
df.isna().sum()

Unnamed: 0                    0
Clothing ID                   0
Age                           0
Title                      3810
Review Text                   0
Rating                        0
Recommended IND               0
Positive Feedback Count       0
Division Name                14
Department Name              14
Class Name                   14
dtype: int64

<h3>- Define target(y) and feature(X)</h3>

In [80]:
df.columns

Index(['Unnamed: 0', 'Clothing ID', 'Age', 'Title', 'Review Text', 'Rating',
       'Recommended IND', 'Positive Feedback Count', 'Division Name',
       'Department Name', 'Class Name'],
      dtype='object')

In [81]:
#............................................................
df.replace({'Rating': { 1:0, 2:0, 3:0, 4:1, 5:1 }}, inplace=True)
y = df['Rating']
x = df['Review Text']
df['Rating'].value_counts()

Rating
1    18208
0     5278
Name: count, dtype: int64

<h3>- Train_test_split</h3>

In [83]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=2529)

In [84]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((16440,), (7046,), (16440,), (7046,))

<h3> - Get Feature Test Conversion to Tokens</h3>

In [86]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(lowercase=True, analyzer='word', ngram_range=(2, 3), stop_words='english', max_features=50000)
x_train = cv.fit_transform(x_train)
cv.get_feature_names_out()

array(['00 big', '00 dresses', '00 fit', ..., 'zipper zip',
       'zippered pockets', 'zippers buttons'], dtype=object)

In [87]:
x_train.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [88]:
x_test = cv.fit_transform(x_test)


In [89]:
cv.get_feature_names_out()

array(['00 fits', '00 fits perfectly', '00 petite', ..., 'zipper little',
       'zipper pockets', 'zipper wouldn'], dtype=object)

In [90]:
x_test.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

<h3>- Get Model Train</h3>

In [92]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()

In [93]:
model.fit(x_train,y_train)

MultinomialNB()

<h3>- Model Prediction</h3>

In [95]:
y_pred = model.predict(x_test)
y_pred

array([1, 0, 1, ..., 0, 1, 1], dtype=int64)

In [96]:
y_pred.shape

(7046,)

<h3>- Get Probability of Each Predicted Class</h3>

In [98]:
model.predict_proba(x_test)

array([[0.00681789, 0.99318211],
       [0.95449908, 0.04550092],
       [0.00480856, 0.99519144],
       ...,
       [0.82969833, 0.17030167],
       [0.07093417, 0.92906583],
       [0.4807814 , 0.5192186 ]])

<h3>- Model Accuracy </h3>

In [100]:
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report

In [101]:
confusion_matrix(y_pred,y_test)

array([[ 727, 2476],
       [ 803, 3040]], dtype=int64)

In [102]:
accuracy_score(y_pred,y_test)

0.5346295770650015

In [103]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.48      0.23      0.31      3203
           1       0.55      0.79      0.65      3843

    accuracy                           0.53      7046
   macro avg       0.51      0.51      0.48      7046
weighted avg       0.52      0.53      0.49      7046

